In [127]:
import pandas as pd
import os
from sqlalchemy import create_engine
import datetime
from tqdm.notebook import tqdm
import numpy as np

## EXTRACT
Store datasets into DataFrames

In [128]:
csv_data = "../webscraping/miami_cl_webscrape.csv"
miami_cl_df = pd.read_csv(csv_data)

In [129]:
miami_cl_df

,Unnamed: 0,datetimes,hoods,prices,bedrooms,bathrooms,sqft,housing_type,laundry,parking,cats,dogs,furnished,titles,descriptions,links
0,0,2021-05-30 14:02,(Dadeland 2/2),"$1,900",2BR,2Ba,none,apartment,w/d in unit,carport,none,none,none,●► 2/2 DADELAND: AMAZING UNIT Unique opportun...,\n\nQR Code Link to This Post\n\n\nPROP # 07K8...,https://miami.craigslist.org/mdc/apa/d/2-dadel...
1,1,2021-05-30 14:02,(Doral),"$2,557",2BR,2Ba,none,apartment,w/d in unit,attached garage,none,none,none,DEPÓSITO DE $ 500,\n\nQR Code Link to This Post\n\n\n*** UNFURNI...,https://miami.craigslist.org/mdc/apa/d/miami-d...
2,2,2021-05-30 14:02,(Miami),"$2,241",2BR,2Ba,1106,apartment,w/d in unit,off-street parking,yes,yes,none,2 Bed 2 Bathroom Apartment,"\n\nQR Code Link to This Post\n\n\nOne, Two, a...",https://miami.craigslist.org/mdc/apa/d/miami-b...
3,3,2021-05-30 14:02,(Doral),"$3,399",3BR,2Ba,none,apartment,w/d in unit,attached garage,none,none,none,"$500 deposit one,two,and three bedroom 24 hour...",\n\nQR Code Link to This Post\n\n\n*** UNFURNI...,https://miami.craigslist.org/mdc/apa/d/miami-5...
4,4,2021-05-30 14:01,(doral),"$1,978",1BR,1Ba,none,apartment,w/d in unit,attached garage,none,none,none,un mes GRATIS,\n\nQR Code Link to This Post\n\n\n*** UNFURNI...,https://miami.craigslist.org/mdc/apa/d/miami-u...
5,5,2021-05-30 14:01,(Doral),"$2,257",2BR,2Ba,none,apartment,w/d in unit,street parking,none,none,none,Depósito de $ 500,\n\nQR Code Link to This Post\n\n\n*** UNFURNI...,https://miami.craigslist.org/mdc/apa/d/miami-d...
6,6,2021-05-30 14:01,(Doral),"$1,978",1BR,1Ba,none,apartment,w/d in unit,attached garage,none,none,none,"mudanza rápida en 1, 2 y 3 habitaciones",\n\nQR Code Link to This Post\n\n\n*** UNFURNI...,https://miami.craigslist.org/mdc/apa/d/miami-m...
7,7,2021-05-30 14:01,(Doral),"$2,557",2BR,2Ba,none,apartment,w/d in unit,attached garage,none,none,none,Apartamentos nuevos extranjeros bienvenidos,\n\nQR Code Link to This Post\n\n\n*** UNFURNI...,https://miami.craigslist.org/mdc/apa/d/miami-a...
8,8,2021-05-30 14:01,(Doral),"$2,257",2BR,2Ba,none,apartment,w/d in unit,attached garage,none,none,none,UN MES GRATIS,\n\nQR Code Link to This Post\n\n\n*** UNFURNI...,https://miami.craigslist.org/mdc/apa/d/miami-u...
9,9,2021-05-30 14:01,(North Miami Beach),"$1,350",1BR,1Ba,600,apartment,laundry on site,valet parking,none,none,none,Newly Renovated 1 Bedroom / 1 Bath,\n\nQR Code Link to This Post\n\n\nNewly renov...,https://miami.craigslist.org/mdc/apa/d/miami-n...


## TRANSFORM
Clean DataFrame

In [130]:
miami_df = miami_cl_df.drop(columns=["Unnamed: 0"])
miami_df.head()

,datetimes,hoods,prices,bedrooms,bathrooms,sqft,housing_type,laundry,parking,cats,dogs,furnished,titles,descriptions,links
0,2021-05-30 14:02,(Dadeland 2/2),"$1,900",2BR,2Ba,none,apartment,w/d in unit,carport,none,none,none,●► 2/2 DADELAND: AMAZING UNIT Unique opportun...,\n\nQR Code Link to This Post\n\n\nPROP # 07K8...,https://miami.craigslist.org/mdc/apa/d/2-dadel...
1,2021-05-30 14:02,(Doral),"$2,557",2BR,2Ba,none,apartment,w/d in unit,attached garage,none,none,none,DEPÓSITO DE $ 500,\n\nQR Code Link to This Post\n\n\n*** UNFURNI...,https://miami.craigslist.org/mdc/apa/d/miami-d...
2,2021-05-30 14:02,(Miami),"$2,241",2BR,2Ba,1106,apartment,w/d in unit,off-street parking,yes,yes,none,2 Bed 2 Bathroom Apartment,"\n\nQR Code Link to This Post\n\n\nOne, Two, a...",https://miami.craigslist.org/mdc/apa/d/miami-b...
3,2021-05-30 14:02,(Doral),"$3,399",3BR,2Ba,none,apartment,w/d in unit,attached garage,none,none,none,"$500 deposit one,two,and three bedroom 24 hour...",\n\nQR Code Link to This Post\n\n\n*** UNFURNI...,https://miami.craigslist.org/mdc/apa/d/miami-5...
4,2021-05-30 14:01,(doral),"$1,978",1BR,1Ba,none,apartment,w/d in unit,attached garage,none,none,none,un mes GRATIS,\n\nQR Code Link to This Post\n\n\n*** UNFURNI...,https://miami.craigslist.org/mdc/apa/d/miami-u...


In [131]:
miami_df = miami_df.rename(columns = {"datetimes": "datetime", "hoods": "neighborhood", "prices": "rent",\
                                                "bedrooms": "br", "bathrooms": "ba", "housing_type": "housingType",\
                                                "titles": "title", "descriptions": "description", "links": "link"})
miami_df.head()

,datetime,neighborhood,rent,br,ba,sqft,housingType,laundry,parking,cats,dogs,furnished,title,description,link
0,2021-05-30 14:02,(Dadeland 2/2),"$1,900",2BR,2Ba,none,apartment,w/d in unit,carport,none,none,none,●► 2/2 DADELAND: AMAZING UNIT Unique opportun...,\n\nQR Code Link to This Post\n\n\nPROP # 07K8...,https://miami.craigslist.org/mdc/apa/d/2-dadel...
1,2021-05-30 14:02,(Doral),"$2,557",2BR,2Ba,none,apartment,w/d in unit,attached garage,none,none,none,DEPÓSITO DE $ 500,\n\nQR Code Link to This Post\n\n\n*** UNFURNI...,https://miami.craigslist.org/mdc/apa/d/miami-d...
2,2021-05-30 14:02,(Miami),"$2,241",2BR,2Ba,1106,apartment,w/d in unit,off-street parking,yes,yes,none,2 Bed 2 Bathroom Apartment,"\n\nQR Code Link to This Post\n\n\nOne, Two, a...",https://miami.craigslist.org/mdc/apa/d/miami-b...
3,2021-05-30 14:02,(Doral),"$3,399",3BR,2Ba,none,apartment,w/d in unit,attached garage,none,none,none,"$500 deposit one,two,and three bedroom 24 hour...",\n\nQR Code Link to This Post\n\n\n*** UNFURNI...,https://miami.craigslist.org/mdc/apa/d/miami-5...
4,2021-05-30 14:01,(doral),"$1,978",1BR,1Ba,none,apartment,w/d in unit,attached garage,none,none,none,un mes GRATIS,\n\nQR Code Link to This Post\n\n\n*** UNFURNI...,https://miami.craigslist.org/mdc/apa/d/miami-u...


In [132]:
miami_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2599 entries, 0 to 2598
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   datetime      2599 non-null   object
 1   neighborhood  2599 non-null   object
 2   rent          2599 non-null   object
 3   br            2599 non-null   object
 4   ba            2599 non-null   object
 5   sqft          2599 non-null   object
 6   housingType   2599 non-null   object
 7   laundry       2599 non-null   object
 8   parking       2599 non-null   object
 9   cats          2599 non-null   object
 10  dogs          2599 non-null   object
 11  furnished     2599 non-null   object
 12  title         2599 non-null   object
 13  description   2599 non-null   object
 14  link          2599 non-null   object
dtypes: object(15)
memory usage: 304.7+ KB


In [133]:
miami_df['datetime']

0       2021-05-30 14:02
1       2021-05-30 14:02
2       2021-05-30 14:02
3       2021-05-30 14:02
4       2021-05-30 14:01
5       2021-05-30 14:01
6       2021-05-30 14:01
7       2021-05-30 14:01
8       2021-05-30 14:01
9       2021-05-30 14:01
10      2021-05-30 14:00
11      2021-05-30 13:58
12      2021-05-30 13:54
13      2021-05-30 13:54
14      2021-05-30 13:54
15      2021-05-30 13:53
16      2021-05-30 13:53
17      2021-05-30 13:51
18      2021-05-30 13:49
19      2021-05-30 13:48
20      2021-05-30 13:44
21      2021-05-30 13:43
22      2021-05-30 13:43
23      2021-05-30 13:38
24      2021-05-30 13:34
25      2021-05-30 13:34
26      2021-05-30 13:34
27      2021-05-30 13:34
28      2021-05-30 13:33
29      2021-05-30 13:33
30      2021-05-30 13:28
31      2021-05-30 13:28
32      2021-05-30 13:27
33      2021-05-30 13:23
34      2021-05-30 13:23
35      2021-05-30 13:22
36      2021-05-30 13:19
37      2021-05-30 13:18
38      2021-05-30 13:15
39      2021-05-30 13:14


In [134]:
miami_df['datetime'] = pd.to_datetime(miami_df['datetime'])
miami_df['datetime']

0      2021-05-30 14:02:00
1      2021-05-30 14:02:00
2      2021-05-30 14:02:00
3      2021-05-30 14:02:00
4      2021-05-30 14:01:00
5      2021-05-30 14:01:00
6      2021-05-30 14:01:00
7      2021-05-30 14:01:00
8      2021-05-30 14:01:00
9      2021-05-30 14:01:00
10     2021-05-30 14:00:00
11     2021-05-30 13:58:00
12     2021-05-30 13:54:00
13     2021-05-30 13:54:00
14     2021-05-30 13:54:00
15     2021-05-30 13:53:00
16     2021-05-30 13:53:00
17     2021-05-30 13:51:00
18     2021-05-30 13:49:00
19     2021-05-30 13:48:00
20     2021-05-30 13:44:00
21     2021-05-30 13:43:00
22     2021-05-30 13:43:00
23     2021-05-30 13:38:00
24     2021-05-30 13:34:00
25     2021-05-30 13:34:00
26     2021-05-30 13:34:00
27     2021-05-30 13:34:00
28     2021-05-30 13:33:00
29     2021-05-30 13:33:00
30     2021-05-30 13:28:00
31     2021-05-30 13:28:00
32     2021-05-30 13:27:00
33     2021-05-30 13:23:00
34     2021-05-30 13:23:00
35     2021-05-30 13:22:00
36     2021-05-30 13:19:00
3

In [135]:
miami_df['neighborhood']

0                                          (Dadeland 2/2)
1                                                 (Doral)
2                                                 (Miami)
3                                                 (Doral)
4                                                 (doral)
5                                                 (Doral)
6                                                 (Doral)
7                                                 (Doral)
8                                                 (Doral)
9                                     (North Miami Beach)
10                                                (Doral)
11                                                (MIAMI)
12                                         (Dadeland 2/2)
13                                         (Hammocks 2/2)
14                                          (Kendall 2/2)
15                                          (NORTH MIAMI)
16                                     (Fontainebleu 2/2)
17            

In [136]:
miami_df['neighborhood'].value_counts()

 (Kendall 2/2)                                                     239
 (Miami Beach)                                                     188
 (Miami)                                                           154
 (SUNNY ISLES BEACH)                                               148
 (Fontainebleu 2/2)                                                 98
 (Doral)                                                            77
 (Dadeland 2/2)                                                     75
 (Coral Gables)                                                     69
 (Hammocks 2/2)                                                     69
 (MIAMI)                                                            53
 (Brickell)                                                         48
 (North Miami)                                                      46
 (DORAL)                                                            43
 (Kendall)                                                          40
 (Miam

In [137]:
pd.set_option('display.max_rows', None)
miami_df['neighborhood'] = miami_df['neighborhood'].str.title()
miami_df['neighborhood'].value_counts()

 (Kendall 2/2)                                                     239
 (Miami)                                                           220
 (Miami Beach)                                                     199
 (Sunny Isles Beach)                                               158
 (Doral)                                                           128
 (Coral Gables)                                                     98
 (Fontainebleu 2/2)                                                 98
 (Brickell)                                                         78
 (Dadeland 2/2)                                                     75
 (Hammocks 2/2)                                                     69
 (North Miami)                                                      62
 (Kendall)                                                          46
 (Miami Beach , South Beach)                                        36
 (North Miami Beach)                                                35
 (Sout

In [138]:
miami_df['neighborhood'] = miami_df['neighborhood'].str.replace('(', '')
miami_df['neighborhood'] = miami_df['neighborhood'].str.replace(')', '')
miami_df['neighborhood'] = miami_df['neighborhood'].str.replace(' 2/2.5', '')
miami_df['neighborhood'] = miami_df['neighborhood'].str.replace(' 2/2', '')
miami_df['neighborhood'] = miami_df['neighborhood'].str.replace(' 2/2', '')
miami_df['neighborhood'] = miami_df['neighborhood'].str.replace('Miami Beach , South Beach', 'South Beach')
miami_df['neighborhood'] = miami_df['neighborhood'].str.replace('South Beach Miami Beach', 'South Beach')
miami_df['neighborhood'] = miami_df['neighborhood'].str.replace('South Dadeland', 'Dadeland')
miami_df['neighborhood'] = miami_df['neighborhood'].str.replace('Edgewater Miami', 'Midtown / Edgewater')
miami_df['neighborhood'] = miami_df['neighborhood'].str.replace(', FL', '')
miami_df['neighborhood'] = miami_df['neighborhood'].str.replace(', Fl', '')
miami_df['neighborhood'] = miami_df['neighborhood'].str.replace('Brickell Miami', 'Brickell')
miami_df['neighborhood'] = miami_df['neighborhood'].str.replace('Miami South Beach', 'South Beach')
miami_df['neighborhood'] = miami_df['neighborhood'].str.replace('Hammocks', 'Kendall')
miami_df['neighborhood'] = miami_df['neighborhood'].str.replace('South Kendall', 'Kendall')
miami_df['neighborhood'] = miami_df['neighborhood'].str.replace('Downtown / Brickell', 'Brickell')
miami_df['neighborhood'] = miami_df['neighborhood'].str.replace('Kendall Lake', 'Kendall')
miami_df['neighborhood'] = miami_df['neighborhood'].str.replace('Deerwod', 'Deerwood')
miami_df['neighborhood'] = miami_df['neighborhood'].str.replace('Downtown Miami', 'Downtown')
miami_df['neighborhood'] = miami_df['neighborhood'].str.replace('Downtown / Miami River', 'Miami River')
miami_df['neighborhood'] = miami_df['neighborhood'].str.replace('Edgewater Miami', 'Edgewater')
miami_df['neighborhood'] = miami_df['neighborhood'].str.replace('Midtown / Wynwood', 'Wynwood')
miami_df['neighborhood'] = miami_df['neighborhood'].str.replace('808 Brickell Key Dr #3105', 'Brickell')
miami_df['neighborhood'] = miami_df['neighborhood'].str.replace('Kkendall', 'Kendall')
miami_df['neighborhood'] = miami_df['neighborhood'].str.replace('Edgewater / Midtown', 'Midtown / Edgewater')
miami_df['neighborhood'] = miami_df['neighborhood'].str.replace('West Hialeah', 'Hialeah')
miami_df['neighborhood'] = miami_df['neighborhood'].str.replace('Aventura - Sunny Isles Beach', 'Sunny Isles Beach')
miami_df['neighborhood'] = miami_df['neighborhood'].str.replace('Little Havana / Dowtowwn', 'Little Havana')
miami_df['neighborhood'] = miami_df['neighborhood'].str.replace('Sunny Isles Beach - Aventura - Oceanfront Building', 'Sunny Isles Beach')
miami_df['neighborhood'] = miami_df['neighborhood'].str.replace('150 Sunny Isles Blvd #1-1102', 'Sunny Isles Beach')
miami_df['neighborhood'] = miami_df['neighborhood'].str.replace('Bricekll', 'Brickell')
miami_df['neighborhood'] = miami_df['neighborhood'].str.replace('Fontainebleu', 'Fontainebleau')
miami_df['neighborhood'].value_counts()

 Kendall                                                         423
 Miami                                                           231
 Miami Beach                                                     204
 Sunny Isles Beach                                               167
 Dadeland                                                        130
 Doral                                                           128
 Fontainebleau                                                   125
 Coral Gables                                                    123
 Brickell                                                        101
 South Beach                                                      68
 North Miami                                                      62
 Downtown                                                         40
 North Miami Beach                                                35
 Hialeah                                                          28
 Aventura                         

In [141]:
# drop rows where neighborhood is outside of miami area

non_miami = ["Pembroke Pines", "Hollywood", "Fort Lauderdale", "Hallandale Beach", "Palm Beach Gardens",\
            "North Lauderdale", "Kissimmee"]

for i in tqdm(non_miami): 
    filt = miami_df[miami_df['neighborhood'].str.contains(i)]
    miami_df.drop(index = filt.index, inplace = True)   

  0%|          | 0/7 [00:00<?, ?it/s]

In [142]:
miami_df['neighborhood'].value_counts()

 Kendall                                                         423
 Miami                                                           231
 Miami Beach                                                     204
 Sunny Isles Beach                                               167
 Dadeland                                                        130
 Doral                                                           128
 Fontainebleau                                                   125
 Coral Gables                                                    123
 Brickell                                                        101
 South Beach                                                      68
 North Miami                                                      62
 Downtown                                                         40
 North Miami Beach                                                35
 Hialeah                                                          28
 Aventura                         

In [143]:
filt

,datetime,neighborhood,rent,br,ba,sqft,housingType,laundry,parking,cats,dogs,furnished,title,description,link
416,2021-05-29 20:43:00,Kissimmee,"$1,350",6BR,5Ba,none,apartment,w/d in unit,attached garage,yes,yes,yes,"$1,350/mo6 bd5 ba",\n\nQR Code Link to This Post\n\n\nBrand New 6...,https://miami.craigslist.org/mdc/apa/d/kissimm...


In [144]:
miami_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2569 entries, 0 to 2598
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   datetime      2569 non-null   datetime64[ns]
 1   neighborhood  2569 non-null   object        
 2   rent          2569 non-null   object        
 3   br            2569 non-null   object        
 4   ba            2569 non-null   object        
 5   sqft          2569 non-null   object        
 6   housingType   2569 non-null   object        
 7   laundry       2569 non-null   object        
 8   parking       2569 non-null   object        
 9   cats          2569 non-null   object        
 10  dogs          2569 non-null   object        
 11  furnished     2569 non-null   object        
 12  title         2569 non-null   object        
 13  description   2569 non-null   object        
 14  link          2569 non-null   object        
dtypes: datetime64[ns](1), object(14)
memor

In [ ]:
miami_df.info()

In [ ]:
indexes_to_drop

In [ ]:
hood_col = miami_df['neighborhood'].value_counts().to_frame()
hood_col

In [ ]:
miami_df['rent']

In [ ]:
miami_df['rent'] = miami_df['rent'].str.replace(',', '')
miami_df['rent'] = miami_df['rent'].str.replace('$', '')
miami_df['rent']

In [ ]:
miami_df['rent'] = pd.to_numeric(miami_df['rent'])
miami_df['rent']

In [ ]:
miami_df['br']

In [ ]:
miami_df['br'] = miami_df['br'].str.replace('BR', '')
miami_df['br'] = pd.to_numeric(miami_df['br'])
miami_df['br']

In [ ]:
miami_df['ba']

In [ ]:
miami_df['ba'] = miami_df['ba'].str.replace('sharedBa', '')
miami_df['ba'] = miami_df['ba'].str.replace('Ba', '')
miami_df['ba'] = pd.to_numeric(miami_df['ba'])
miami_df['ba']

In [ ]:
miami_df['sqft']

In [ ]:
miami_df['sqft'] = miami_df['sqft'].str.replace('none', '')
miami_df['sqft'] = pd.to_numeric(miami_df['sqft'])
miami_df['sqft']

In [ ]:
miami_df['housingType'].value_counts()

In [ ]:
miami_df['laundry'].value_counts()

In [ ]:
miami_df['parking'].value_counts()

In [ ]:
miami_df['cats'].value_counts()

In [ ]:
miami_df['cats'] = miami_df['cats'].str.replace('none', 'no')
miami_df['cats']

In [ ]:
miami_df['dogs'].value_counts()

In [ ]:
miami_df['dogs'] = miami_df['dogs'].str.replace('none', 'no')
miami_df['dogs'].value_counts()

In [ ]:
miami_df['furnished'].value_counts()

In [ ]:
miami_df['furnished'] = miami_df['furnished'].str.replace('none', 'no')
miami_df['furnished'].value_counts()

In [ ]:
miami_df['title']

In [ ]:
miami_df['title'] = miami_df['title'].str.replace('●►  ', '')
miami_df['title'] = miami_df['title'].str.replace('*', '')
miami_df['title'] = miami_df['title'].str.replace('●', '')
miami_df['title']

In [ ]:
miami_df['description']

In [ ]:
miami_df['description'] = miami_df['description'].str.replace('\n\nQR Code Link to This Post\n\n\n', '')
miami_df['description'] = miami_df['description'].str.replace('\n', '')
miami_df['description'] = miami_df['description'].str.replace('PROP # 07K', '')
miami_df['description'] = miami_df['description'].str.replace('PROP # 06K', '')
miami_df['description']

In [ ]:
miami_df['link'].value_counts()